In [1]:
import os

import pandas as pd
from sqlalchemy import create_engine

import mipipe as mip

# MySQL 데이터베이스 연결 정보
username = 'root'
password = os.getenv('AIMED_PW')
host = '172.28.8.103'
port = '3306'  # 예: '3306'
database = "MIMIC_III"
db_engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [2]:
from importlib import reload

reload(mip)  # mymodule을 다시 로드하여 변경 사항 반영.

<module 'mipipe' from 'C:\\Users\\airro\\PycharmProjects\\MIMIC_preprocessing\\mipipe\\__init__.py'>

In [23]:
query = "SELECT * FROM CHARTEVENTS LIMIT 10000;"
icu_patient = pd.read_sql(query, db_engine)
icu_patient

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,788,36,165660,241249,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525,15,15.00,L/min,0,0,None,None
1,789,36,165660,241249,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525,100,100.00,,0,0,None,None
2,790,36,165660,241249,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823,.37,0.37,,0,0,None,None
3,791,36,165660,241249,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823,6,6.00,min,0,0,None,None
4,792,36,165660,241249,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823,2.5,2.50,,0,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,10986,109,176760,256504,220277,2142-01-21 13:00:00,2142-01-21 13:03:00,16643,100,100.00,%,0,0,None,None
9996,10987,109,176760,256504,226531,2142-01-21 13:00:00,2142-01-21 13:02:00,16643,113.3,113.30,,0,0,None,None
9997,10988,109,176760,256504,220045,2142-01-21 14:00:00,2142-01-21 14:11:00,16643,92,92.00,bpm,0,0,None,None
9998,10989,109,176760,256504,220179,2142-01-21 14:00:00,2142-01-21 14:11:00,16643,173,173.00,mmHg,0,0,None,None


In [24]:
t1 = mip.group_variables(icu_patient)
t1

,ICUSTAY_ID,ITEMID,CHARTTIME,VALUENUM
0,222630,220045,2141-12-20 13:00:00,91.0
1,222630,220045,2141-12-20 13:15:00,94.0
2,222630,220045,2141-12-20 14:00:00,91.0
3,222630,220045,2141-12-20 14:15:00,92.0
4,222630,220045,2141-12-20 14:30:00,93.0
...,...,...,...,...
9913,291697,227457,2167-07-26 03:06:00,24.0
9914,291697,227457,2167-07-27 04:38:00,16.0
9915,291697,227465,2167-07-27 04:38:00,13.7
9916,291697,227466,2167-07-27 04:38:00,27.4


In [25]:
t2 = mip.hour_aggregate(t1, [220210, 220045], ["mean"])
t2

,ICUSTAY_ID,CHARTTIME_H,220045_mean,220210_mean
0,222630,0,90.000000,20.0
1,222630,1,97.500000,14.0
2,222630,2,100.333333,16.0
3,222630,3,100.000000,15.5
4,222630,4,97.000000,14.0
...,...,...,...,...
406,291697,19,114.000000,35.0
407,291697,20,102.000000,33.0
408,291697,21,126.333333,36.0
409,291697,22,113.500000,29.0


In [29]:
# read json file
import json

with open('X.feature_names.json') as f:
    x_feature_names = json.load(f)

d_items = pd.read_csv("../mimic3_csv/D_ITEMS.csv")  # D_ITEMS.csv
d_labitems = pd.read_csv("../mimic3_csv/D_LABITEMS.csv")  # D_LABITEMS.csv
x_feature_names

item_map = {}
not_itemid = []

for item_id in x_feature_names:
    id = item_id.split("_")[0]
    # if id is number
    if id.isdigit():
        id = int(id)
        try:
            label = d_items[d_items["ITEMID"] == id]
            item_map[id] = label["LABEL"].values[0]
        except:
            try:
                label = d_labitems[d_labitems["ITEMID"] == id]
                item_map[id] = label["LABEL"].values[0]
            except:
                print("NOT FOUND: ", id)


item_map

{220048: 'Heart Rhythm',
 220046: 'Heart rate Alarm - High',
 220047: 'Heart Rate Alarm - Low',
 220052: 'Arterial Blood Pressure mean',
 220056: 'Arterial Blood Pressure Alarm - Low',
 220058: 'Arterial Blood Pressure Alarm - High',
 220059: 'Pulmonary Artery Pressure systolic',
 220060: 'Pulmonary Artery Pressure diastolic',
 220061: 'Pulmonary Artery Pressure mean',
 220063: 'Pulmonary Artery Pressure Alarm - High',
 220066: 'Pulmonary Artery Pressure Alarm - Low',
 220072: 'Central Venous Pressure Alarm - High',
 220073: 'Central Venous Pressure  Alarm - Low',
 220074: 'Central Venous Pressure',
 220088: 'Cardiac Output (thermodilution)',
 220120: 'Intra Aortic Ballon Pump Setting',
 220181: 'Non Invasive Blood Pressure mean',
 220194: 'Temporary AV interval',
 220292: 'Minute Volume Alarm - Low',
 220293: 'Minute Volume Alarm - High',
 220339: 'PEEP set',
 220739: 'GCS - Eye Opening',
 220765: 'Intra Cranial Pressure',
 220862: 'Albumin 25%',
 220864: 'Albumin 5%',
 220949: 'Dextr

In [16]:
for i in not_itemid:
    if "220045" in i:
        print(i)

In [42]:
not_itemid

['DiaBP',
 'HR',
 'RR',
 'SpO2',
 'SysBP',
 '220048: 1st AV (First degree AV Block) ',
 '220048: 2nd AV M2 (Second degree AV Block - Mobitz 2) ',
 '220048: 2nd AV W-M1 (Second degree AV Block Wenckebach - Mobitz1) ',
 '220048: 3rd AV (Complete Heart Block) ',
 '220048: A Flut (Atrial Flutter) ',
 '220048: A Paced',
 '220048: AF (Atrial Fibrillation)',
 '220048: AV Paced',
 '220048: Idioventricular',
 '220048: JR (Junctional Rhythm)',
 '220048: JT (Junctional Tachycardia) ',
 '220048: LBBB (Left Bundle Branch Block) ',
 '220048: MAT (Multifocal atrial tachycardia)',
 '220048: PAT (Paroxysmal Atrial Tachycardia)',
 '220048: RBBB (Right Bundle Branch Block) ',
 '220048: SA (Sinus Arrhythmia)',
 '220048: SB (Sinus Bradycardia)',
 '220048: SR (Sinus Rhythm)',
 '220048: ST (Sinus Tachycardia) ',
 '220048: SVT (Supra Ventricular Tachycardia)',
 '220048: V Paced',
 '220048: VF (Ventricular Fibrillation) ',
 '220048: VT (Ventricular Tachycardia) ',
 '220048: WAP (Wandering atrial pacemaker)',
 

In [ ]:
import pandas as pd
import multiprocessing as mp

def process_group(group):
    # 그룹 데이터 처리 함수
    return group.mean()  # 예시로 그룹의 평균값을 계산

def parallel_process_dataframe(df):
    with mp.Pool(mp.cpu_count()) as pool:
        result = pool.map(process_group, [group for name, group in df.groupby('ID')])
    return pd.concat(result)

# 예시 데이터프레임
df = pd.DataFrame({
    'ID': [1, 1, 2, 2, 3, 3],
    'value': [10, 20, 30, 40, 50, 60]
})

result_df = parallel_process_dataframe(df)
print(result_df)
